In [1]:
import os

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
# from sklearn.model_selection import GridSearchCV

from pyspark.sql import SparkSession
import pyspark
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
import pyspark.pandas as ps

In [3]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled",True)
ps.set_option("compute.default_index_type", "distributed")

In [4]:
spark

In [5]:
df =  pd.read_csv("5w_dummies.csv",encoding='utf-8-sig')
# df

In [6]:
df = df.drop(['嘉義市.1'], axis=1)
df = df.drop(['ASP.NET'], axis=1)
df = df.drop(['新竹市.1'], axis=1)
df = df.drop(['C++.Net'], axis=1)
df = df.drop(['D3.js'], axis=1)
df = df.drop(['Node.js'], axis=1)
df = df.drop(['Visual-Basic-.net'], axis=1)
df = df.drop(['Visual-Studio-.net'], axis=1)
df = df.drop(['X.25'], axis=1)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df, test_size=0.1, random_state=0)

In [8]:
feature_names = X_train.columns
feature_names = feature_names.tolist()
train_data = X_train.loc[:,feature_names]
train_data['label'] = train_data['工作待遇_min']
train_data = ps.DataFrame(train_data)
train_data = train_data.to_spark()
feature_cols = feature_names
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
train_data = assembler.transform(train_data)
train_model = train_data.select(['features', 'label'])
train_model.show(10)


/opt/spark-3.3.1-bin-hadoop3/python/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/spark-3.3.1-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]
/opt/spark-3.3.1-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


23/01/04 07:37:24 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/01/04 07:37:24 WARN TaskSetManager: Stage 0 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|            features|label|
+--------------------+-----+
|(761,[0,2,3,5,7,2...|32000|
|(761,[0,2,3,5,7,1...|33000|
|(761,[0,2,3,4,8,1...|40000|
|(761,[0,2,3,5,7,2...|40000|
|(761,[0,2,3,5,7,2...|53846|
|(761,[0,2,3,5,7,1...|35000|
|(761,[0,2,3,5,8,2...|50000|
|(761,[0,2,3,5,7,1...|32000|
|(761,[0,1,2,3,5,8...|30000|
|(761,[0,2,3,5,8,2...|33000|
+--------------------+-----+
only showing top 10 rows



In [9]:
# df3 = df.sample(2000, random_state=0)
test_data = X_test.loc[:, feature_names]
test_data['label'] = test_data['工作待遇_min']
test_data = ps.DataFrame(test_data)
test_data = test_data.to_spark()
feature_cols2 = feature_names
assembler = VectorAssembler(inputCols=feature_cols2, outputCol='features')
test_data = assembler.transform(test_data)
test_model = test_data.select(['features', 'label'])
test_model.show(10)

/opt/spark-3.3.1-bin-hadoop3/python/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/spark-3.3.1-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]
/opt/spark-3.3.1-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


+--------------------+------+
|            features| label|
+--------------------+------+
|(761,[0,2,3,5,7,2...| 34000|
|(761,[0,2,3,5,7,2...| 33000|
|(761,[0,2,3,5,7,3...| 36000|
|(761,[0,2,3,5,9,2...|100000|
|(761,[0,2,3,5,7,1...| 37000|
|(761,[0,2,3,5,7,2...| 32000|
|(761,[0,2,3,5,7,2...| 60000|
|(761,[0,2,3,5,7,2...| 33100|
|(761,[0,2,3,5,8,1...| 27000|
|(761,[0,2,3,4,8,2...| 25250|
+--------------------+------+
only showing top 10 rows



In [109]:
lin_Reg=LinearRegression(labelCol='label', elasticNetParam=1 , regParam=0.01)

In [110]:
lr_model=lin_Reg.fit(train_model)

23/01/03 12:25:30 WARN TaskSetManager: Stage 11 contains a task of very large size (1840 KiB). The maximum recommended task size is 1000 KiB.


23/01/03 12:25:38 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/01/03 12:25:38 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/01/03 12:25:38 WARN TaskSetManager: Stage 13 contains a task of very large size (1840 KiB). The maximum recommended task size is 1000 KiB.


In [111]:
lr_model.intercept

135.93976482007272

In [112]:
lr_model.coefficients

DenseVector([0.0, 0.1995, -0.0057, 1.0, -0.2127, 0.2127, 16.1694, 16.6437, 16.9187, 17.0062, 32.1005, 39.8233, 307.5604, 160.2768, 209.9109, 24.1587, 129.6585, -28.457, -38.6305, 59.4049, 0.0, 35.4595, -5.0499, 19.0224, 18.2292, 239.4674, 43.6664, -49.6146, 55.2046, 478.8978, -1.0329, -4.1209, -112.1544, -57.5367, -208.8074, -50.324, 25.821, -57.2073, -57.6719, -72.955, -57.4793, -300.0443, -62.6645, -112.9865, -112.6924, -47.963, -144.8149, -208.6358, -57.2284, -47.7663, -112.4026, -3.7428, -50.4676, -144.4818, 0.0, -48.2673, -112.8149, 0.0, -112.9106, -62.3572, -57.2206, 25.5668, 0.0, -209.1323, -50.2085, -50.3714, -69.5828, -73.7281, -112.7109, -176.2293, -68.3694, -73.5355, 0.0, -112.862, -208.8147, 23.0593, -27.9476, 0.6312, -72.3362, -72.4502, -144.3682, -208.4605, -144.8347, -48.071, -57.5717, -145.738, 0.0, -144.4796, -112.9953, -207.8716, -26.5486, -75.0746, -112.7994, -62.3036, -113.1261, -50.3212, -144.6226, -298.7831, -50.0128, -62.4135, -73.4526, -51.2651, -72.0911, -73.11

In [113]:
training_predictions=lr_model.evaluate(train_model)

23/01/03 12:26:17 WARN TaskSetManager: Stage 15 contains a task of very large size (1840 KiB). The maximum recommended task size is 1000 KiB.


In [114]:
training_predictions.r2

0.9999999954481541

In [15]:
%%timeit
lin_Reg=LinearRegression(labelCol='label', elasticNetParam=1 , regParam=50 ,maxIter=200000)
lr_model=lin_Reg.fit(train_model)
# print(lr_model.intercept)
# print(lr_model.coefficients)
training_predictions=lr_model.evaluate(train_model)


23/01/03 13:55:54 WARN TaskSetManager: Stage 52 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


23/01/03 13:55:56 WARN TaskSetManager: Stage 54 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


23/01/03 13:55:58 WARN TaskSetManager: Stage 56 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


0.9999959095063032
2499.9914565385216
23/01/03 13:55:59 WARN TaskSetManager: Stage 58 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


23/01/03 13:56:01 WARN TaskSetManager: Stage 60 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


...

23/01/03 13:56:27 WARN TaskSetManager: Stage 92 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


0.9999959095063032
2499.991456538521
23/01/03 13:56:28 WARN TaskSetManager: Stage 94 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


23/01/03 13:56:30 WARN TaskSetManager: Stage 96 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


23/01/03 13:56:31 WARN TaskSetManager: Stage 98 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


0.9999959095063032
2499.9914565385216
4.74 s ± 211 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
S_RFC = RandomForestRegressor(maxDepth=20) #numTrees=100,
 
RFC_model = S_RFC.fit(train_model)

y_pred_RFC = RFC_model.transform(test_model)

eval_r2 = RegressionEvaluator(metricName='r2')
eval_mse = RegressionEvaluator(metricName='mse')
rsquare = eval_r2.evaluate(y_pred_RFC)
mse = eval_mse.evaluate(y_pred_RFC)

# print(rsquare)
# print(mse)

23/01/04 07:37:46 WARN TaskSetManager: Stage 2 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


23/01/04 07:37:48 WARN TaskSetManager: Stage 3 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


23/01/04 07:37:56 WARN TaskSetManager: Stage 4 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


23/01/04 07:38:00 WARN TaskSetManager: Stage 6 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


23/01/04 07:38:02 WARN TaskSetManager: Stage 8 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


...

23/01/04 07:45:34 WARN DAGScheduler: Broadcasting large task binary with size 11.1 MiB
23/01/04 07:45:35 WARN TaskSetManager: Stage 374 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


23/01/04 07:45:38 WARN DAGScheduler: Broadcasting large task binary with size 11.7 MiB
23/01/04 07:45:38 WARN TaskSetManager: Stage 376 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


23/01/04 07:45:41 WARN DAGScheduler: Broadcasting large task binary with size 12.1 MiB
23/01/04 07:45:41 WARN TaskSetManager: Stage 378 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


23/01/04 07:45:45 WARN DAGScheduler: Broadcasting large task binary with size 12.4 MiB
23/01/04 07:45:45 WARN TaskSetManager: Stage 380 contains a task of very large size (1661 KiB). The maximum recommended task size is 1000 KiB.


0.9644943303264242
31061420.50576415
57.7 s ± 789 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
